In [3]:
import numpy as np
from toolbox import kinematic_mapping as km
import pandas as pd



In [197]:
class grid():   
    
    def __init__(self, x_min=-1, x_max=1, y_min=-1, y_max=1, z_min=0, z_max=1, grid_width=0.1):
        self.xs = np.arange(x_min, x_max, grid_width)
        self.ys = np.arange(y_min, y_max, grid_width)
        self.zs = np.arange(z_min, z_max, grid_width)        
        
    def is_in_grid(self, point): 
        idxs = np.zeros(3)
        
        if (point[0]<self.xs[0]) or (point[0]>self.xs[-1]): 
            return (False, None)
        else: 
            idxs[0] = np.argwhere(point[0]<self.xs)[0][0]
            
        if (point[1]<self.ys[0]) or (point[1]>self.ys[-1]): 
            return (False, None)
        else: idxs[1] = np.argwhere(point[1]<self.ys)[0][0]
        
        if (point[2]<self.zs[0]) or (point[2]>self.zs[-1]): 
            return (False, None)
        else: 
            idxs[2] = np.argwhere(point[2]<self.zs)[0][0]
            
        return (True, idxs.astype(int))
        

joint_id = [5,5,5,5,5,4,3,3,3]
lidar_x = [0,-7,0,7,0,0,8,14,0]
lidar_y = [-8,5,4,5,12,0,5,5,0]
lidar_z = [0,0,6,0,0,12,6,-1,-12]
point_x = [0,-1,0,1,0,0,0,0,0]
point_y = [-1,0,0,0,1,0,0,1,0]
point_z = [0,0,1,0,0,1,1,0,-2]

d = {'joint_id':joint_id, 'lidar_x':lidar_x, 'lidar_y':lidar_y, 
     'lidar_z':lidar_z, 'point_x':point_x, 'point_y':point_y, 'point_z':point_z}

df = pd.DataFrame(d)

In [196]:

class localizer(): 
    def __init__(self): 
        self.angle_df = km.init_params()
        
        self.lidarDF = self.init_lidar_constants()
        #joint the lidar is attached to
        self.lidar_joint = {str(self.lidarDF['lidar_id'][i]):self.lidarDF['joint_id'][i] for i in range(len(self.lidarDF))}
        
        self.lidar_base = {str(self.lidarDF['lidar_id'][i]): 
                           0.01*np.array([float(self.lidarDF['lidar_x'][i]), self.lidarDF['lidar_y'][i], self.lidarDF['lidar_z'][i]])
                           for i in range(len(self.lidarDF))}
        
        self.lidar_direction = {str(self.lidarDF['lidar_id'][i]): 
                           np.array([float(self.lidarDF['point_x'][i]), self.lidarDF['point_y'][i], self.lidarDF['point_z'][i]])
                           for i in range(len(self.lidarDF))}
        self.world_grid = grid()
    
    def init_lidar_constants(self): 
        lidar_id = [0,1,2,3,4,5,6,7,8]
        joint_id = [5,5,5,5,5,4,3,3,4]
        lidar_x = [0,-7,0,7,0,0,8,14,0]
        lidar_y = [-8,5,4,5,12,0,5,5,0]
        lidar_z = [0,0,6,0,0,12,6,-1,-12]
        point_x = [0,-1,0,1,0,0,0,0,0]
        point_y = [-1,0,0,0,1,0,0,1,0]
        point_z = [0,0,1,0,0,1,1,0,-2]

        d = {'lidar_id':lidar_id, 'joint_id':joint_id, 'lidar_x':lidar_x, 'lidar_y':lidar_y, 
         'lidar_z':lidar_z, 'point_x':point_x, 'point_y':point_y, 'point_z':point_z}

        return pd.DataFrame(d)       
    
    def create_transformation(self, joint_angles): 
        self.angle_df['theta'].values[0:7] = joint_angles
        TF_Base = km.Transform2Base(self.angle_df)
        return TF_Base
        
    def localize_joints(self, joint_angles):         
        TF_Base = self.create_transformation(joint_angles)
        joint_base = np.array([0,0,0,1])
        joints = [km.ExecuteTransform(TF_Base[i], joint_base) for i in range(len(TF_Base))]
        return np.array(joints)[:,:3]
    
    def localize_lidars(self, joint_angles): 
        TF_Base = self.create_transformation(joint_angles)   
        lidar_poses = []
        for lid in range(9): 
            joint = self.lidar_joint[str(lid)]
            lidar_tmp = np.ones(4)
            lidar_tmp[:3] = self.lidar_base[str(lid)]
            lidar_poses.append(km.ExecuteTransform(TF_Base[joint-1], lidar_tmp))
        return np.array(lidar_poses)[:,:3]

    
    def localize_obstacle(self, joint_pos, lidar_measurement, lidar_index):
        measurement = lidar_measurement[lidar_index]
        distance = np.min([measurement, 2000])/1000. # clip and transform to meters 
        location_joint = self.lidar_base[str(lidar_index)] + distance*self.lidar_direction[str(lidar_index)]
        location_joint_tmp = np.ones(4)
        location_joint_tmp[:3] = location_joint
        TF_Base = self.create_transformation(joint_pos)
        
        location = km.ExecuteTransform(TF_Base[self.lidar_joint[str(lidar_index)]], location_joint_tmp)[:3]
        self.identify_area(location)
        return location

    
    def identify_area(self, location):         
        x_dir = ['back', 'front']
        y_dir = ['right','left']
        z_dir = ['bottom', 'top']        
        comparison = np.array([0, 0, 0.4])
        select = (location>comparison).astype(int)       
        print('obstacle found in {} {} {}'.format(x_dir[select[0]], y_dir[select[1]], z_dir[select[2]]))

        
    def predict_collision(self, obstacle_location, future_joint_angels): 
        #future_joint_angels = joints x timestamps
        T = future_joint_angels.shape[1]  #number of future timestamps
        J = future_joint_angels.shape[0] 
        obstacle_in_grid = self.world_grid.is_in_grid(obstacle_location)        
        if obstacle_in_grid[0] == False: 
            print('Obstacle outside world')
            return        
        else: 
            for t in range(T): 
                joint_poses = self.localize_joints(future_joint_angels[:, t])
                grid_locs = [self.world_grid.is_in_grid(joint_poses[j, :])[1] for j in range(J)]
                collision = np.array([(grid_loc == obstacle_in_grid[1]).all() for grid_loc in grid_locs])
                if collision.any(): 
                    print('possible collision in {} steps'.format(t))
                    return (True, t)
                    
            

In [184]:
loc = localizer()

In [185]:
loc.localize_joints(np.ones(7))

array([[0.   , 0.   , 0.333],
       [0.   , 0.   , 0.333],
       [0.144, 0.224, 0.504],
       [0.098, 0.282, 0.466],
       [0.363, 0.122, 0.708],
       [0.363, 0.122, 0.708],
       [0.39 , 0.128, 0.792],
       [0.3  , 0.18 , 0.816]])

In [186]:
loc.localize_lidars(np.ones(7)).shape

(9, 3)

In [187]:
loc.localize_obstacle(np.ones(7), np.ones(9)*1222, 5)

obstacle found in front right top


array([ 1.315, -0.156,  1.613])

In [193]:
a = loc.predict_collision(np.array([0, 0, 0.35]), np.ones((7, 9)))

[array([11, 11,  4]), array([11, 11,  4]), array([12, 13,  6]), array([11, 13,  5]), array([14, 12,  8]), array([14, 12,  8]), array([14, 12,  8])] [11 11  4]
[ True  True False False False False False]


In [194]:
a

(True, 0)

In [195]:
a = np.ones(9)*1222
a[5]

1222.0

In [157]:
np.min([1,2])

1

In [158]:
def grid_area(x_min=-1, x_max=-1, y_min=-1, y_max=-1, z_min=0, z_max=-1, grid_width=0.1): 
    xs = np.arange(x_min, x_max, grid_width)
    ys = np.arange(y_min, y_max, grid_width)
    zs = np.arange(z_min, z_max, grid_width)
    
    return xs, ys, zs


In [143]:
a = np.arange(0, 1, 0.1)
print(a)
aa = 1

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [144]:
gr = grid()

In [145]:
gr.zs

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [91]:
a = gr.is_in_grid(np.array([0.1, 0.2, 0.1]))

In [95]:
(a[1] == a[1]).all()

True

In [98]:
b = np.array([True, False, True])

In [100]:
b.any()

True